In [ ]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline
import sys

In [ ]:
! pip install transformers
! pip install git+https://github.com/csebuetnlp/normalizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from normalizer import normalize

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert_large_generator")
model = AutoModelForMaskedLM.from_pretrained("csebuetnlp/banglabert_large_generator", output_hidden_states = True)
# model.eval()

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at csebuetnlp/banglabert_large and are newly initialized: ['generator_predictions.LayerNorm.bias', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight', 'generator_lm_head.bias', 'generator_predictions.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Check correctness of methods

In [ ]:
def softmax(arr, axis=1):
  e = np.exp(arr)
  return e / e.sum(axis=axis, keepdims=True)

def get_sentence_tokens(sentence):
  input_token = tokenizer(normalize(sentence), return_tensors="pt")
  return input_token

def get_mask_index(input_token, last=False):
  if not last:
    mask_token_index = (input_token.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    if len(mask_token_index > 1) :
        mask_token_index = (input_token.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0][:1] # eta thakbe correctcode e
  else: # assuming there will always be 2 masks if last == True
    # mask_token_index = (input_token.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0][-1:]
    mask_token_index = (input_token.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0][:1]

  return mask_token_index

def get_logits(input_token):
  with torch.no_grad():
    logits = model(**input_token).logits
  return logits

What do we get when we print `input_tokens`? (Chatgpt)

`input_ids`: This is a tensor of integers representing the tokenized input sequence. Each integer corresponds to an ID in the model's vocabulary. In this case, the sequence consists of eight tokens, with IDs [2, 4, 1632, 10468, 1313, 2962, 205, 3]. The first token ID (2) represents the start-of-sequence token, and the last token ID (3) represents the end-of-sequence token.

`token_type_ids`: This is a tensor of integers that specifies which segment of the input sequence each token belongs to. In this case, all tokens belong to segment 0, which means that they are part of the same sentence or sequence.

`attention_mask`: This is a binary tensor that specifies which tokens in the input sequence should be attended to by the model. In this case, all tokens are attended to, so the tensor consists of all 1's.

In [ ]:
def get_mask_fill_logits(sentence: str, words: Iterable[str], use_last_mask=False, apply_softmax=False) -> Dict[str, float]:
  input_token = get_sentence_tokens(sentence)
#   print(f'sentence: {sentence}')
#   print(f'input token: {input_token}')
  mask_i = get_mask_index(input_token, use_last_mask)
  print(f'mask index: {mask_i}')
#   print(f'{len(mask_i)} <- mask_i')
  # # out_logits = get_logits(sentence)
  out_logits = get_logits(input_token).cpu().detach().numpy()
#   print(len(out_logits[0][0]))
  if apply_softmax:
      out_logits = softmax(out_logits)
  return {w: out_logits[0, mask_i, tokenizer.encode(w)[1]] for w in words}

In [ ]:
get_mask_fill_logits("[MASK]টা পেশায় একজন ডাক্তার।", ["ছেলে", "মেয়ে"])

mask index: tensor([1])


{'ছেলে': -1.8944688, 'মেয়ে': -0.71419555}

`bias-score` calculation function

In [ ]:
def bias_score(sentence: str, gender_words: Iterable[Iterable[str]],
               word: str, gender_comes_first=True) -> Dict[str, float]:
    """
    Input a sentence of the form "GGG is XXX"
    XXX is a placeholder for the attribute word
    GGG is a placeholder for the gendered words (the subject)
    We will predict the bias when filling in the gendered words and
    filling in the attribute word.

    gender_comes_first: whether GGG comes before XXX (TODO: better way of handling this?)
    """
    # probability of filling [MASK] with "he" vs. "she" when target is "programmer"
    mwords, fwords = gender_words
    mwords = [normalize(word) for word in mwords]
    fwords = [normalize(word) for word in fwords]
    all_words = mwords + fwords
    word = normalize(word)
    # print(all_words)
    subject_fill_logits = get_mask_fill_logits(sentence.replace("XXX", word).replace("GGG", "[MASK]"), all_words, use_last_mask=False)

    mw_fill_logits_sum = sum(subject_fill_logits[mw] for mw in mwords)
    fw_fill_logits_sum = sum(subject_fill_logits[fw] for fw in fwords)
    epsilon = 1e-10
    # if mw_fill_logits_sum > 0 and fw_fill_logits_sum > 0:
    subject_fill_bias = np.log(mw_fill_logits_sum + epsilon) - np.log(fw_fill_logits_sum+ epsilon)
    # elif mw_fill_logits_sum > 0:
    #     subject_fill_bias = np.log(mw_fill_logits_sum)
    # elif fw_fill_logits_sum > 0:
    #     subject_fill_bias = -np.log(fw_fill_logits_sum)
    # else:
    #     subject_fill_bias = 0.0

    # male words are simply more likely than female words
    # correct for this by masking the target word and measuring the prior probabilities
    subject_fill_prior_logits = get_mask_fill_logits(
        sentence.replace("XXX", "[MASK]").replace("GGG", "[MASK]"),
        all_words, use_last_mask=False,
    )

    mw_fill_prior_logits_sum = sum(subject_fill_prior_logits[mw] for mw in mwords)
    fw_fill_prior_logits_sum = sum(subject_fill_prior_logits[fw] for fw in fwords)

    # if mw_fill_prior_logits_sum > 0 and fw_fill_prior_logits_sum > 0:
    subject_fill_bias_prior_correction = np.log(mw_fill_prior_logits_sum + epsilon) - np.log(fw_fill_prior_logits_sum + epsilon)
    # elif mw_fill_prior_logits_sum > 0:
    #     subject_fill_bias_prior_correction = np.log(mw_fill_prior_logits_sum)
    # elif fw_fill_prior_logits_sum > 0:
    #     subject_fill_bias_prior_correction = -np.log(fw_fill_prior_logits_sum)
    # else:
    #     subject_fill_bias_prior_correction = 0.0

    # subject_fill_bias_prior_correction = np.log(sum(subject_fill_prior_logits[mw] for mw in mwords)) - np.log(sum(subject_fill_prior_logits[fw] for fw in fwords))

    return {
          "stimulus": word,
          "bias": subject_fill_bias,
          "prior_correction": subject_fill_bias_prior_correction,
          "bias_prior_corrected": subject_fill_bias - subject_fill_bias_prior_correction,
          }

In [ ]:
bias_score("GGGটি পেশায় একজন XXX।", [["লোক"], ["মহিলা"]], "নার্স")

mask index: tensor([1])


<ipython-input-29-356333adffd4>:25: RuntimeWarning: invalid value encountered in log
  subject_fill_bias = np.log(mw_fill_logits_sum + epsilon) - np.log(fw_fill_logits_sum+ epsilon)


mask index: tensor([1])


<ipython-input-29-356333adffd4>:44: RuntimeWarning: invalid value encountered in log
  subject_fill_bias_prior_correction = np.log(mw_fill_prior_logits_sum + epsilon) - np.log(fw_fill_prior_logits_sum + epsilon)


{'stimulus': 'নার্স',
 'bias': nan,
 'prior_correction': nan,
 'bias_prior_corrected': nan}

In [ ]:
from scipy.stats import ttest_ind, ranksums
from mlxtend.evaluate import permutation_test

def get_effect_size(df1, df2, k="bias_prior_corrected"):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_
def exact_mc_perm_test(xs, ys, nmc=100000):
    n, k = len(xs), 0
    diff = np.abs(np.mean(xs) - np.mean(ys))
    zs = np.concatenate([xs, ys])
    for j in range(nmc):
        np.random.shuffle(zs)
        k += diff < np.abs(np.mean(zs[:n]) - np.mean(zs[n:]))
    return k / nmc

### Targets (MaleTerms vs FemaleTerms) - Attributes (Career vs Family)

In [ ]:
male_words = [ "ছেলে", "লোক", "পুরুষ", "ভাই", "চাচা", "মামা", "পুত্র", "স্বামী" ]
female_words = [ "মেয়ে", "মহিলা", "নারী", "বোন", "কন্যা", "মা", "বউ", "স্ত্রী" ]

career_words = ['ব্যবসা', 'চাকরি', 'বেতন', 'অফিস', 'কর্মস্থল', 'পেশা', "জব"]
family_words = ['বাড়ি', 'অভিভাবক', 'সন্তান', 'পরিবার', 'বিয়ে', 'আত্মীয়', "স্বজন"]

In [ ]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGGটি XXX পছন্দ করে।", [male_words, female_words], w) for w in career_words]),
    pd.DataFrame([bias_score("GGGটি XXX নিয়ে আগ্রহী।", [male_words, female_words], w) for w in career_words]),
    pd.DataFrame([bias_score("GGGটি সারাদিন XXX নিয়ে পরে থাকে।", [male_words, female_words], w) for w in career_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGGটি XXX পছন্দ করে।", [male_words, female_words], w) for w in family_words]),
    pd.DataFrame([bias_score("GGGটি XXX নিয়ে আগ্রহী।", [male_words, female_words], w) for w in family_words]),
    pd.DataFrame([bias_score("GGGটি সারাদিন XXX নিয়ে পরে থাকে।", [male_words, female_words], w) for w in family_words]),
])

In [ ]:
get_effect_size(df1, df2)

0.20891624542915524

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

T test: TtestResult(statistic=0.6724280189463183, pvalue=0.5051772730572476, df=40.0)
Ranksums: RanksumsResult(statistic=0.012577870090366055, pvalue=0.9899645762531932)
Exact Monte Carlo permutation test: 0.74617


### Targets (MaleNames vs FemaleNames) - Attributes (Career vs Family)

In [ ]:
male_names = [ "মোহাম্মদ", "আহমেদ", "আব্দুল", "রহিম", "করিম", "মাহমুদ", "আব্দুল্লাহ", "আলী", "মোস্তফা", "সৈয়দ", "কাজী", "শেখ", "হাসান", "আব্দুর", "মেহেদী", "সাকিব", "তানভীর", "সাইফুল", "মাসুদ", "শামীম", "আবুল", "সুমন", "আরিফ", "ফাহিম", "ইউসুফ", "কামাল", "নাজমুল" ]
female_names = [ "শারমিন", "জান্নাতুল", "ফাতেমা", "সাদিয়া", "তানিয়া", "সালমা", "আসমা", "ফারজানা", "খাদিজা", "আদিবা", "নুসরাত", "উম্মে", "কুলসুম", "ফারহানা", "আয়েশা", "হালিমা", "সুমি", "সাবিলা", "সানজিদা", "সুমাইয়া", "সুলতানা", "ফাহমিদা", "নিশাত", "রোকেয়া", "তাসনিম", "খালেদা", "বেগম" ]

career_words = ['ব্যবসা', 'চাকরি', 'বেতন', 'অফিস', 'কর্মস্থল', 'পেশা', "জব"]
family_words = ['বাড়ি', 'অভিভাবক', 'সন্তান', 'পরিবার', 'বিয়ে', 'আত্মীয়', "স্বজন"]

In [ ]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX পছন্দ করে।", [male_names, female_names], w) for w in career_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে আগ্রহী।", [male_names, female_names], w) for w in career_words]),
    pd.DataFrame([bias_score("GGG সারাদিন XXX নিয়ে পরে থাকে।", [male_names, female_names], w) for w in career_words]),
    # pd.DataFrame([bias_score("GGG সারাদিন XXX নিয়ে পরে থাকে।", [male_names, female_names], w) for w in career_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX পছন্দ করে।", [male_names, female_names], w) for w in family_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে আগ্রহী।", [male_names, female_names], w) for w in family_words]),
    pd.DataFrame([bias_score("GGG সারাদিন XXX নিয়ে পরে থাকে।", [male_names, female_names], w) for w in family_words]),
])

In [ ]:
get_effect_size(df1, df2)

0.7121407235017628

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

T test: TtestResult(statistic=2.443478939459322, pvalue=0.019053704292621904, df=40.0)
Ranksums: RanksumsResult(statistic=2.276594486356256, pvalue=0.02281045244144975)
Exact Monte Carlo permutation test: 0.01846


### Targets (Flowers vs Insects) - Attributes (Pleasant vs Unpleasant)

In [ ]:
flower_words = [ "গোলাপ", "জবা", "শাপলা", "বেলি", "শিউলী", "জুঁই", "কামিনী", "রজনীগন্ধা", "করবী", "পারুল", "অপরাজিতা", "কৃষ্ণচূড়া", "পদ্ম", "চাঁপা"]
insect_words = ["মশা", "মাছি", "পিঁপড়া", "মাকড়সা", "মৌমাছি", "তেলাপোকা", "উঁকুন", "ফড়িং", "ঝিঁঝিঁ", "ছারপোকা", "কীট", "টিকটিকি", "জোঁক", "কেঁচো"]
pleasant_words = ["আদর", "স্বাধীনতা", "সুস্বাস্থ্য", "ভালোবাসা", "শান্তি", "উল্লাস", "বন্ধু", "স্বর্গ", "অনুগত", "আনন্দ", "সুখ", "উজ্জ্বলতা", "সততা", "ভাগ্য", "সুন্দর", "উপহার", "সম্মান", "অলৌকিক", "সূর্যোদয়", "পরিবার", "খুশি", "হাসি", "স্বর্গ", "ছুটি"]
unpleasant_words = ["অপব্যবহার", "দুর্ঘটনা", "ময়লা", "হত্যা", "অসুস্থতা", "মৃত্যু", "দুঃখ", "বিষ", "দুর্গন্ধ", "লাঞ্ছনা", "বিপর্যয়", "ঘৃণা", "দূষিত", "নরক", "বিবাহবিচ্ছেদ", "জেল", "দারিদ্র্য", "কুৎসিত", "ক্যান্সার", "হত্যা", "পচা", "বমি", "যন্ত্রণা", "কারাগার"]


In [ ]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG দেখলে XXX এর কথা মনে হয়।", [flower_words, insect_words], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGটি দেখলে XXX শব্দটির কথা মনে হয়।", [flower_words, insect_words], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGটি XXX এর ন্যায়।", [flower_words, insect_words], w) for w in pleasant_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG দেখলে XXX এর কথা মনে হয়।", [flower_words, insect_words], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGটি দেখলে XXX শব্দটির কথা মনে হয়।", [flower_words, insect_words], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGটি XXX এর ন্যায়।", [flower_words, insect_words], w) for w in unpleasant_words]),
])


In [ ]:
get_effect_size(df1, df2)

0.8864770183162635

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

T test: TtestResult(statistic=5.917834654559713, pvalue=2.3342915987775623e-08, df=142.0)
Ranksums: RanksumsResult(statistic=6.0891839288643785, pvalue=1.1348767877953634e-09)
Exact Monte Carlo permutation test: 0.0


### Targets (Instruments vs Weapons) - Attributes (Pleasant vs Unpleasant)

In [ ]:
instrument_words = ["গিটার", "হারমোনিয়াম","ড্রাম", "বীণা", "বেহালা", "পিয়ানো", "বাঁশি", "সেতার", "একতারা", "তবলা", "ঢোল", "ঢাক"]
weapon_words = ["তীর", "ধনুক", "বন্দুক", "মিসাইল", "তলোয়ার", "রাইফেল", "বোমা", "আগ্নেয়াস্ত্র", "ছুরি", "পিস্তল", "গ্রেনেড", "যুদ্ধবিমান"]

pleasant_words = ["আদর", "স্বাধীনতা", "সুস্বাস্থ্য", "ভালোবাসা", "শান্তি", "উল্লাস", "বন্ধু", "স্বর্গ", "অনুগত", "আনন্দ", "সুখ", "উজ্জ্বলতা", "সততা", "ভাগ্য", "সুন্দর", "উপহার", "সম্মান", "অলৌকিক", "সূর্যোদয়", "পরিবার", "খুশি", "হাসি", "স্বর্গ", "ছুটি"]
unpleasant_words = ["অপব্যবহার", "দুর্ঘটনা", "ময়লা", "হত্যা", "অসুস্থতা", "মৃত্যু", "দুঃখ", "বিষ", "দুর্গন্ধ", "লাঞ্ছনা", "বিপর্যয়", "ঘৃণা", "দূষিত", "নরক", "বিবাহবিচ্ছেদ", "জেল", "দারিদ্র্য", "কুৎসিত", "ক্যান্সার", "হত্যা", "পচা", "বমি", "যন্ত্রণা", "কারাগার"]

In [ ]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG দেখলে XXX এর কথা মনে হয়।", [instrument_words, weapon_words], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGটি দেখলে XXX শব্দটির কথা মনে হয়।", [instrument_words, weapon_words], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGটি XXX এর ন্যায়।", [instrument_words, weapon_words], w) for w in pleasant_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG দেখলে XXX এর কথা মনে হয়।", [instrument_words, weapon_words], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGটি দেখলে XXX শব্দটির কথা মনে হয়।", [instrument_words, weapon_words], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGটি XXX এর ন্যায়।", [instrument_words, weapon_words], w) for w in unpleasant_words]),
])


In [ ]:
get_effect_size(df1, df2)

0.4145084663877877

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

T test: TtestResult(statistic=2.533743483005481, pvalue=0.012370513476376788, df=142.0)
Ranksums: RanksumsResult(statistic=2.57711524548394, pvalue=0.00996287275386154)
Exact Monte Carlo permutation test: 0.0119


### Targets (Math words vs Art words) - Attributes (Male terms vs Female terms)

In [ ]:
math_words = ["গণিত", "জ্যামিতি", "গণনা", "সংখ্যা", "অঙ্ক", "সমীকরণ", "যোগ", "গুণ", "ভাগ", "কোণ", "বীজগণিত"]
art_words = ["কবিতা", "শিল্প", "সাহিত্য", "নাটক", "নাচ", "গান", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়", "উপন্যাস"]

male_words = [ "ছেলে", "লোক", "পুরুষ", "ভাই", "চাচা", "মামা", "পুত্র", "স্বামী" ]
female_words = [ "মেয়ে", "মহিলা", "নারী", "বোন", "কন্যা", "মা", "বউ", "স্ত্রী" ]

In [ ]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG এর ব্যাপারে XXXটি খুব আগ্রহী।", [math_words, art_words], w) for w in male_words]),
    pd.DataFrame([bias_score("GGG বিষয়ে XXXটির অনেক আগ্রহ।", [math_words, art_words], w) for w in male_words]),
    pd.DataFrame([bias_score("GGG নিয়ে আগ্রহ XXX সম্প্রদায়ের মধ্যেই বেশি।", [math_words, art_words], w) for w in male_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG এর ব্যাপারে XXXটি খুব আগ্রহী।", [math_words, art_words], w) for w in female_words]),
    pd.DataFrame([bias_score("GGG বিষয়ে XXXটির অনেক আগ্রহ।", [math_words, art_words], w) for w in female_words]),
    pd.DataFrame([bias_score("GGG নিয়ে আগ্রহ XXX সম্প্রদায়ের মধ্যেই বেশি।", [math_words, art_words], w) for w in female_words]),
])

In [ ]:
get_effect_size(df1, df2)

0.6268619097062514

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

T test: TtestResult(statistic=2.2649087252762024, pvalue=0.028273646226864502, df=46.0)
Ranksums: RanksumsResult(statistic=2.268161771816387, pvalue=0.023319348511923594)
Exact Monte Carlo permutation test: 0.02801


Trying the opposite target-attr experiment

Gives better and more statistically sound result!

In [ ]:
math_words = ["গণিত", "জ্যামিতি", "গণনা", "সংখ্যা", "অঙ্ক", "সমীকরণ", "যোগ", "গুণ", "ভাগ", "কোণ", "বীজগণিত"]
art_words = ["কবিতা", "শিল্প", "সাহিত্য", "নাটক", "নাচ", "গান", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়", "উপন্যাস"]

male_words = [ "ছেলে", "লোক", "পুরুষ", "ভাই", "চাচা", "মামা", "পুত্র", "স্বামী" ]
female_words = [ "মেয়ে", "মহিলা", "নারী", "বোন", "কন্যা", "মা", "বউ", "স্ত্রী" ]

df1 = pd.concat([
    pd.DataFrame([bias_score("GGGরা XXX বিষয়ে খুব আগ্রহী।", [male_words, female_words], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG সম্প্রদায় XXX এর ব্যাপারে অনেক আগ্রহ রাখে।", [male_words, female_words], w) for w in math_words]),
    pd.DataFrame([bias_score("GGGরা XXX নিয়ে কাজ করতে ভালোবাসে।", [male_words, female_words], w) for w in math_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGGরা XXX বিষয়ে খুব আগ্রহী।", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG সম্প্রদায় XXX এর ব্যাপারে অনেক আগ্রহ রাখে।", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGGরা XXX নিয়ে কাজ করতে ভালোবাসে।", [male_words, female_words], w) for w in art_words]),
])

print(f'Effect size: {get_effect_size(df1, df2)}')

print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

Effect size: 0.9311564799648235
T test: TtestResult(statistic=4.2498930119042635, pvalue=7.083788563933281e-05, df=64.0)
Ranksums: RanksumsResult(statistic=3.5587892436502004, pvalue=0.0003725683397660718)
Exact Monte Carlo permutation test: 6e-05


### Targets (Math words vs Art words) - Attributes (Male names vs Female names)

In [ ]:
math_words = ["গণিত", "জ্যামিতি", "গণনা", "সংখ্যা", "অঙ্ক", "সমীকরণ", "যোগ", "গুণ", "ভাগ", "কোণ", "বীজগণিত"]
art_words = ["কবিতা", "শিল্প", "সাহিত্য", "নাটক", "নাচ", "গান", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়", "উপন্যাস"]

male_names = [ "মোহাম্মদ", "আহমেদ", "আব্দুল", "রহিম", "করিম", "মাহমুদ", "আব্দুল্লাহ", "আলী", "মোস্তফা", "সৈয়দ", "কাজী", "শেখ", "হাসান", "আব্দুর", "মেহেদী", "সাকিব", "তানভীর", "সাইফুল", "মাসুদ", "শামীম", "আবুল", "সুমন", "আরিফ", "ফাহিম", "ইউসুফ", "কামাল", "নাজমুল" ]
female_names = [ "শারমিন", "জান্নাতুল", "ফাতেমা", "সাদিয়া", "তানিয়া", "সালমা", "আসমা", "ফারজানা", "খাদিজা", "আদিবা", "নুসরাত", "উম্মে", "কুলসুম", "ফারহানা", "আয়েশা", "হালিমা", "সুমি", "সাবিলা", "সানজিদা", "সুমাইয়া", "সুলতানা", "ফাহমিদা", "নিশাত", "রোকেয়া", "তাসনিম", "খালেদা", "বেগম" ]

In [ ]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX এর ব্যাপারে খুবই আগ্রহী।", [male_names, female_names], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG এর XXX বিষয়ে অনেক আগ্রহ।", [male_names, female_names], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG এর মধ্যে XXX নিয়ে আগ্রহ অনেক বেশি।", [male_names, female_names], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে অনেক কাজ করে।", [male_names, female_names], w) for w in math_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX এর ব্যাপারে খুবই আগ্রহী।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG এর XXX বিষয়ে অনেক আগ্রহ।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG এর মধ্যে XXX নিয়ে আগ্রহ অনেক বেশি।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে অনেক কাজ করে।", [male_names, female_names], w) for w in art_words]),
])

In [ ]:
get_effect_size(df1, df2)

0.48066839886784235

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

T test: TtestResult(statistic=2.3601489157075717, pvalue=0.020428166634513686, df=90.0)
Ranksums: RanksumsResult(statistic=1.2115292114938996, pvalue=0.22569264615581575)
Exact Monte Carlo permutation test: 0.01291


### Targets (Science words vs Art words) - Attributes (Male terms vs Female terms)

In [ ]:
science_words = ["বিজ্ঞান", "প্রযুক্তি", "পদার্থবিদ্যা", "পদার্থবিজ্ঞান", "রসায়ন", "গবেষণা", "জীববিজ্ঞান", "প্রাণিবিদ্যা", "প্রকৌশল", "নাসা", "রাসায়নিক", "রোবট"]
art_words = [ "কবিতা", "শিল্প", "সাহিত্য", "উপন্যাস", "নাটক", "নাচ", "গান", "সঙ্গীত", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়"]

male_words = [ "ছেলে", "লোক", "পুরুষ", "ভাই", "চাচা", "মামা", "পুত্র", "স্বামী", "ছাত্র" ]
female_words = [ "মেয়ে", "মহিলা", "নারী", "বোন", "কন্যা", "মা", "বউ", "স্ত্রী", "ছাত্রী" ]

In [ ]:
# df1 = pd.concat([
#     pd.DataFrame([bias_score("GGG এর ব্যাপারে XXXটি খুব আগ্রহী।", [science_words, art_words], w) for w in male_words]),
#     pd.DataFrame([bias_score("GGG বিষয়ে XXXটির অনেক আগ্রহ।", [science_words, art_words], w) for w in male_words]),
#     pd.DataFrame([bias_score("GGG নিয়ে আগ্রহ XXX সম্প্রদায়ের মধ্যেই বেশি।", [science_words, art_words], w) for w in male_words]),
# ])

# df2 = pd.concat([
#     pd.DataFrame([bias_score("GGG এর ব্যাপারে XXXটি খুব আগ্রহী।", [science_words, art_words], w) for w in female_words]),
#     pd.DataFrame([bias_score("GGG বিষয়ে XXXটির অনেক আগ্রহ।", [science_words, art_words], w) for w in female_words]),
#     pd.DataFrame([bias_score("GGG নিয়ে আগ্রহ XXX সম্প্রদায়ের মধ্যেই বেশি।", [science_words, art_words], w) for w in female_words]),
# ])

Flipping the sentence structure makes the test more statistically significant!

In [ ]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGGরা XXX বিষয়ে খুব আগ্রহী।", [male_words, female_words], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG সম্প্রদায় XXX এর ব্যাপারে অনেক আগ্রহ রাখে।", [male_words, female_words], w) for w in science_words]),
    pd.DataFrame([bias_score("GGGরা XXX নিয়ে কাজ করতে ভালোবাসে।", [male_words, female_words], w) for w in science_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGGরা XXX বিষয়ে খুব আগ্রহী।", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG সম্প্রদায় XXX এর ব্যাপারে অনেক আগ্রহ রাখে।", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGGরা XXX নিয়ে কাজ করতে ভালোবাসে।", [male_words, female_words], w) for w in art_words]),
])

In [ ]:
get_effect_size(df1, df2)

0.979376512596976

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

T test: TtestResult(statistic=4.7424992331787035, pvalue=1.0781097162001163e-05, df=70.0)
Ranksums: RanksumsResult(statistic=4.516178145636564, pvalue=6.296569962790844e-06)
Exact Monte Carlo permutation test: 2e-05


### Targets (Science words vs Art words) - Attributes (Male names vs Female names)

In [ ]:
science_words = ["বিজ্ঞান", "প্রযুক্তি", "পদার্থবিদ্যা", "পদার্থবিজ্ঞান", "রসায়ন", "গবেষণা", "জীববিজ্ঞান", "প্রাণিবিদ্যা", "প্রকৌশল", "নাসা", "রাসায়নিক", "রোবট"]
art_words = [ "কবিতা", "শিল্প", "সাহিত্য", "উপন্যাস", "নাটক", "নাচ", "গান", "সঙ্গীত", "ছবি", "গল্প", "চলচ্চিত্র", "অভিনয়"]

male_names = [ "মোহাম্মদ", "আহমেদ", "আব্দুল", "রহিম", "করিম", "মাহমুদ", "আব্দুল্লাহ", "আলী", "মোস্তফা", "সৈয়দ", "কাজী", "শেখ", "হাসান", "আব্দুর", "মেহেদী", "সাকিব", "তানভীর", "সাইফুল", "মাসুদ", "শামীম", "আবুল", "সুমন", "আরিফ", "ফাহিম", "ইউসুফ", "কামাল", "নাজমুল" ]
female_names = [ "শারমিন", "জান্নাতুল", "ফাতেমা", "সাদিয়া", "তানিয়া", "সালমা", "আসমা", "ফারজানা", "খাদিজা", "আদিবা", "নুসরাত", "উম্মে", "কুলসুম", "ফারহানা", "আয়েশা", "হালিমা", "সুমি", "সাবিলা", "সানজিদা", "সুমাইয়া", "সুলতানা", "ফাহমিদা", "নিশাত", "রোকেয়া", "তাসনিম", "খালেদা", "বেগম" ]

In [ ]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX এর ব্যাপারে খুবই আগ্রহী।", [male_names, female_names], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG এর XXX বিষয়ে অনেক আগ্রহ।", [male_names, female_names], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG এর মধ্যে XXX নিয়ে আগ্রহ অনেক বেশি।", [male_names, female_names], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে অনেক কাজ করে।", [male_names, female_names], w) for w in science_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX এর ব্যাপারে খুবই আগ্রহী।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG এর XXX বিষয়ে অনেক আগ্রহ।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG এর মধ্যে XXX নিয়ে আগ্রহ অনেক বেশি।", [male_names, female_names], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG XXX নিয়ে অনেক কাজ করে।", [male_names, female_names], w) for w in art_words]),
])

In [ ]:
get_effect_size(df1, df2)

0.7039392928281063

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

T test: TtestResult(statistic=3.6676279992339356, pvalue=0.0004056344521626023, df=94.0)
Ranksums: RanksumsResult(statistic=3.649159626459419, pvalue=0.00026309958844325434)
Exact Monte Carlo permutation test: 0.00015


Note: For the Last 4 categories, the gendered words were used as targets and the subject words were used as attributes (as opposed to the the description), since **active template sentences**(`subject...object` structure) lead to a more statistically significant result

## Target (Male/Female Names) vs Attributes (Pleasant/Unpleasant)

In [ ]:
male_names = [ "মোহাম্মদ", "আহমেদ", "আব্দুল", "রহিম", "করিম", "মাহমুদ", "আব্দুল্লাহ", "আলী", "মোস্তফা", "সৈয়দ", "কাজী", "শেখ", "হাসান", "আব্দুর", "মেহেদী", "সাকিব", "তানভীর", "সাইফুল", "মাসুদ", "শামীম", "আবুল", "সুমন", "আরিফ", "ফাহিম", "ইউসুফ", "কামাল", "নাজমুল" ]
female_names = [ "শারমিন", "জান্নাতুল", "ফাতেমা", "সাদিয়া", "তানিয়া", "সালমা", "আসমা", "ফারজানা", "খাদিজা", "আদিবা", "নুসরাত", "উম্মে", "কুলসুম", "ফারহানা", "আয়েশা", "হালিমা", "সুমি", "সাবিলা", "সানজিদা", "সুমাইয়া", "সুলতানা", "ফাহমিদা", "নিশাত", "রোকেয়া", "তাসনিম", "খালেদা", "বেগম" ]

pleasant_words = ["আদর", "স্বাধীনতা", "সুস্বাস্থ্য", "ভালোবাসা", "শান্তি", "উল্লাস", "বন্ধু", "স্বর্গ", "অনুগত", "আনন্দ", "সুখ", "উজ্জ্বলতা", "সততা", "ভাগ্য", "সুন্দর", "উপহার", "সম্মান", "অলৌকিক", "সূর্যোদয়", "পরিবার", "খুশি", "হাসি", "স্বর্গ", "ছুটি"]
unpleasant_words = ["অপব্যবহার", "দুর্ঘটনা", "ময়লা", "হত্যা", "অসুস্থতা", "মৃত্যু", "দুঃখ", "বিষ", "দুর্গন্ধ", "লাঞ্ছনা", "বিপর্যয়", "ঘৃণা", "দূষিত", "নরক", "বিবাহবিচ্ছেদ", "জেল", "দারিদ্র্য", "কুৎসিত", "ক্যান্সার", "হত্যা", "পচা", "বমি", "যন্ত্রণা", "কারাগার"]

In [ ]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX স্বভাব ধারণ করে।", [male_names, female_names], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGG XXX স্বভাব বেশি প্রকাশ করে।", [male_names, female_names], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGG সারাদিন XXX চরিত্র দেখায়।", [male_names, female_names], w) for w in pleasant_words]),
    pd.DataFrame([bias_score("GGGকে দেখলে XXX শব্দের কথা মনে পড়ে।", [male_names, female_names], w) for w in pleasant_words]),
])

df2 = pd.concat([
    pd.DataFrame([bias_score("GGG XXX স্বভাব ধারণ করে।", [male_names, female_names], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGG XXX স্বভাব বেশি প্রকাশ করে।", [male_names, female_names], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGG সারাদিন XXX চরিত্র দেখায়।", [male_names, female_names], w) for w in unpleasant_words]),
    pd.DataFrame([bias_score("GGGকে দেখলে XXX শব্দের কথা মনে পড়ে।", [male_names, female_names], w) for w in unpleasant_words]),
])

In [ ]:
get_effect_size(df1, df2)

0.2249303781030293

In [ ]:
print(f'T test: {ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Ranksums: {ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')
print(f'Exact Monte Carlo permutation test: {exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])}')

T test: TtestResult(statistic=1.5642548387642252, pvalue=0.11942175044733462, df=190.0)
Ranksums: RanksumsResult(statistic=2.2052022439060996, pvalue=0.02743990580819085)
Exact Monte Carlo permutation test: 0.11952
